In [10]:
import os
import pandas as pd

# O diretório atual onde o script Python está localizado
pasta = os.getcwd()

# Dicionário para armazenar os DataFrames por ano
dfs_por_ano = {}

# Percorra todos os arquivos no diretório atual
for arquivo in os.listdir(pasta):
    if arquivo.endswith('.csv'):  # Verifique se o arquivo é CSV
        # Extraia o ano do nome do arquivo (assumindo formato "ca-ano-semestre.csv")
        ano = arquivo.split('-')[1]  # Pega o ano no nome do arquivo
        
        # Crie o caminho completo para o arquivo
        caminho_arquivo = os.path.join(pasta, arquivo)
        
        # Tente ler o arquivo com a codificação ISO-8859-1 (ou latin1)
        try:
            df = pd.read_csv(caminho_arquivo, delimiter=";", encoding="ISO-8859-1")
        except UnicodeDecodeError:
            # Caso ocorra erro de codificação, tente uma codificação alternativa
            df = pd.read_csv(caminho_arquivo, delimiter=";", encoding="latin1")
        
        # Se o ano ainda não foi adicionado ao dicionário, crie uma nova lista
        if ano not in dfs_por_ano:
            dfs_por_ano[ano] = []
        
        # Adicione o DataFrame à lista correspondente ao ano
        dfs_por_ano[ano].append(df)

# Agora, para cada ano, combine os DataFrames
for ano, dfs_ano in dfs_por_ano.items():
    # Combine os DataFrames do mesmo ano
    df_combinado_ano = pd.concat(dfs_ano, ignore_index=True)
    
    # Salve em um novo arquivo CSV para cada ano
    caminho_saida = os.path.join(pasta, f'dados_combinados_{ano}.csv')
    df_combinado_ano.to_csv(caminho_saida, index=False)
    
    print(f"Arquivos do ano {ano} foram combinados com sucesso!")


C:\Users\kaio\AppData\Local\Temp\ipykernel_20500\3240734137.py:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=";", encoding="ISO-8859-1")


Arquivos do ano 2019 foram combinados com sucesso!
Arquivos do ano 2020 foram combinados com sucesso!
Arquivos do ano 2021 foram combinados com sucesso!
Arquivos do ano 2022 foram combinados com sucesso!
Arquivos do ano 2023 foram combinados com sucesso!
Arquivos do ano 2024 foram combinados com sucesso!


In [12]:
import os
import pandas as pd

# O diretório atual onde o script Python está localizado
pasta = os.getcwd()

# Lista para armazenar todos os DataFrames
todos_dfs = []

# Percorra todos os arquivos no diretório atual
for arquivo in os.listdir(pasta):
    if arquivo.endswith('.csv'):  # Verifique se o arquivo é CSV
        # Crie o caminho completo para o arquivo
        caminho_arquivo = os.path.join(pasta, arquivo)
        
        # Tente ler o arquivo com a codificação ISO-8859-1 (ou latin1)
        try:
            df = pd.read_csv(caminho_arquivo, delimiter=",", encoding="ISO-8859-1")
        except UnicodeDecodeError:
            # Caso ocorra erro de codificação, tente uma codificação alternativa
            df = pd.read_csv(caminho_arquivo, delimiter=",", encoding="latin1")
        
        # Adicione o DataFrame à lista de todos os DataFrames
        todos_dfs.append(df)

# Combine todos os DataFrames em um único DataFrame
df_combinado = pd.concat(todos_dfs, ignore_index=True)

# Salve o DataFrame combinado em um novo arquivo CSV
caminho_saida = os.path.join(pasta, 'dados_combinados_geral.csv')
df_combinado.to_csv(caminho_saida, index=False)

print("Todos os arquivos CSV foram combinados com sucesso em 'dados_combinados_geral.csv'!")


C:\Users\kaio\AppData\Local\Temp\ipykernel_20500\3779154639.py:18: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=",", encoding="ISO-8859-1")
C:\Users\kaio\AppData\Local\Temp\ipykernel_20500\3779154639.py:18: DtypeWarning: Columns (0,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=",", encoding="ISO-8859-1")


Todos os arquivos CSV foram combinados com sucesso em 'dados_combinados_geral.csv'!


In [15]:
# Crie um dicionário para armazenar DataFrames separados por município
# Defina os municípios desejados
municipios_desejados = ['FORTALEZA', 'SALVADOR', 'TERESINA', 'JOAO PESSOA', 'SAO LUIS', 'NATAL']

# Filtre o DataFrame para incluir apenas os municípios desejados
df_filtrado = df_combinado[df_combinado['Municipio'].isin(municipios_desejados)]

# Selecione somente as colunas de interesse
df_filtrado = df_filtrado[['Municipio','Produto', 'Data da Coleta', 'Valor de Venda', 'Valor de Compra']]

# Salve o DataFrame filtrado em um novo arquivo CSV
caminho_saida_filtrado = os.path.join(pasta, 'dados_filtrados_municipios.csv')
df_filtrado.to_csv(caminho_saida_filtrado, index=False)

print("Dados filtrados e salvos em 'dados_filtrados_municipios.csv'!")


dfs_municipios = {}

# Para cada município desejado, filtre o DataFrame
for municipio in municipios_desejados:
    # Filtra os dados para o município atual
    df_municipio = df_filtrado[df_filtrado['Municipio'] == municipio]
    
    # Adiciona o DataFrame ao dicionário, usando o nome do município como chave
    dfs_municipios[municipio] = df_municipio
    
    # Salve cada DataFrame em um arquivo CSV
    caminho_saida_municipio = os.path.join(pasta, f'dados_{municipio}.csv')
    df_municipio.to_csv(caminho_saida_municipio, index=False)

print("Dados separados e salvos em arquivos CSV para cada município!")


Dados filtrados e salvos em 'dados_filtrados_municipios.csv'!
Dados separados e salvos em arquivos CSV para cada município!


In [18]:
import os
import pandas as pd

# Definir a pasta onde os arquivos serão salvos
pasta_saida = os.path.join(pasta, 'data')

# Criar a pasta caso não exista
os.makedirs(pasta_saida, exist_ok=True)

# Para cada município, agrupar os dados semanalmente e calcular as médias
for municipio, df_municipio in dfs_municipios.items():
    # Garantir que 'Data da Coleta' seja do tipo datetime
    df_municipio['Data da Coleta'] = pd.to_datetime(df_municipio['Data da Coleta'], errors='coerce')
    
    # Definir 'Data da Coleta' para o último dia da semana (domingo)
    df_municipio['Data da Coleta'] = df_municipio['Data da Coleta'] + pd.to_timedelta(6 - df_municipio['Data da Coleta'].dt.weekday, unit='D')
    
    # Agrupar por semana e calcular a média para 'Valor de Venda' e 'Valor de Compra'
    df_semanal = df_municipio.groupby('Data da Coleta').agg({
        'Produto': 'first',  # Mantém o primeiro produto da semana (supondo que seja o mesmo durante a semana)
        'Valor de Venda': 'mean',
        'Valor de Compra': 'mean'
    }).reset_index()
    
    # Salvar o DataFrame resultante como um CSV na pasta 'data'
    caminho_saida = os.path.join(pasta_saida, f'{municipio}.week.csv')
    df_semanal.to_csv(caminho_saida, index=False)

print("Arquivos semanais foram salvos com sucesso!")


C:\Users\kaio\AppData\Local\Temp\ipykernel_20500\924127924.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_municipio['Data da Coleta'] = pd.to_datetime(df_municipio['Data da Coleta'], errors='coerce')
C:\Users\kaio\AppData\Local\Temp\ipykernel_20500\924127924.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_municipio['Data da Coleta'] = df_municipio['Data da Coleta'] + pd.to_timedelta(6 - df_municipio['Data da Coleta'].dt.weekday, unit='D')
C:\Users\kaio\AppData\Local\Temp\ipykernel_20500\924

TypeError: agg function failed [how->mean,dtype->object]

In [27]:
import pandas as pd
import os

# Caminho de entrada e saída
pasta_entrada = os.getcwd()  # Substitua pelo caminho onde os dados estão localizados
pasta_saida = os.path.join(pasta_entrada, 'data')  # Substitua pelo caminho para salvar os resultados

# Lista de municípios
municipios = ['dados_FORTALEZA', 'dados_JOAO PESSOA', 'dados_SAO LUIS', 'dados_NATAL', 'dados_SALVADOR', 'dados_TERESINA']  # Substitua pelos municípios que você deseja processar

# Processamento de dados para cada município
for municipio in municipios:
    # Carregar o arquivo CSV
    caminho_entrada = os.path.join(pasta_entrada, f'{municipio}.csv')
    df_municipio = pd.read_csv(caminho_entrada)

    # Substituir vírgulas por pontos e converter as colunas de 'Valor de Venda' e 'Valor de Compra' para float
    df_municipio['Valor de Venda'] = df_municipio['Valor de Venda'].str.replace(',', '.').astype(float)

    # Converter a coluna de 'Data da Coleta' para o tipo datetime
    df_municipio['Data da Coleta'] = pd.to_datetime(df_municipio['Data da Coleta'], format='%d/%m/%Y')

    # Filtrar apenas os dados onde o 'Produto' é igual a 'GASOLINA'
    df_gasolina = df_municipio[df_municipio['Produto'] == 'GASOLINA']

    # Agrupar os dados por semana (começando no domingo) e calcular a média
    df_semanal = df_gasolina.groupby(pd.Grouper(key='Data da Coleta', freq='W-SUN', label='left', closed='left')).agg({
        'Produto': 'first',  # Assume que o produto é o mesmo durante a semana
        'Valor de Venda': 'mean',
    }).reset_index()

    # Salvar o DataFrame resultante como um CSV na pasta de saída
    caminho_saida = os.path.join(pasta_saida, f'{municipio}.week.csv')
    df_semanal.to_csv(caminho_saida, index=False)

    print(f'Arquivo gerado para {municipio}: {caminho_saida}')



Arquivo gerado para dados_FORTALEZA: c:\Users\kaio\OneDrive\Área de Trabalho\Dolar Analyctics\dadobruto\data\dados_FORTALEZA.week.csv
Arquivo gerado para dados_JOAO PESSOA: c:\Users\kaio\OneDrive\Área de Trabalho\Dolar Analyctics\dadobruto\data\dados_JOAO PESSOA.week.csv
Arquivo gerado para dados_SAO LUIS: c:\Users\kaio\OneDrive\Área de Trabalho\Dolar Analyctics\dadobruto\data\dados_SAO LUIS.week.csv
Arquivo gerado para dados_NATAL: c:\Users\kaio\OneDrive\Área de Trabalho\Dolar Analyctics\dadobruto\data\dados_NATAL.week.csv
Arquivo gerado para dados_SALVADOR: c:\Users\kaio\OneDrive\Área de Trabalho\Dolar Analyctics\dadobruto\data\dados_SALVADOR.week.csv
Arquivo gerado para dados_TERESINA: c:\Users\kaio\OneDrive\Área de Trabalho\Dolar Analyctics\dadobruto\data\dados_TERESINA.week.csv
